# Create a dataframe of NTRL Lab Analysis Results month by month

In [ ]:
# This program helps you generate the dataframe for each month per area of sampling

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [2]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

In [3]:
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
    
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df    

In [4]:
#monthyear
currentmonthyr_01 = datetime.date(2020,1,1)
currentmonthyr_02 = datetime.date(2020,2,1)
currentmonthyr_03 = datetime.date(2020,3,1)
currentmonthyr_04 = datetime.date(2020,4,1)
currentmonthyr_05 = datetime.date(2020,5,1)
currentmonthyr_06 = datetime.date(2020,6,1)
currentmonthyr_07 = datetime.date(2020,7,1)


#filepath
filepath_01 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MARCH 2020\3) March _2020.xlsx'
filepath_04 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\APRIL 2020\4) April _2020.xlsx'
filepath_05 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MAY 2020\5) MAY_2020.xlsx'
filepath_06 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JUNE 2020\6) June _2020.xlsx'
filepath_07 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\7) July _2020.xlsx'

#list_lab_index for ntrl
ntrl_Pb = ['B', 'C', 'E', '104_Pb']
ntr_Zn =  ['B', 'C', 'F', '104_Zn']
ntrl_Cu = ['B', 'C', 'G', '104_Cu']
ntrl_Ni = ['B', 'C', 'H', '104_Ni']
ntrl_Co = ['B', 'C', 'I', '104_Co']
ntrl_Mn = ['B', 'C', 'K', '104_Mn']
ntrl_Cr = ['B', 'C', 'L', '104_Cr']
ntrl_Ca = ['B', 'C', 'M', '104_Ca']
ntrl_Si = ['B', 'C', 'N', '104_Si']
ntrl_Al = ['B', 'C', 'O', '104_Al']
ntrl_Mg = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH = ['B', 'C', 'BB', '104_pH60']
ntrl_ORP = ['B', 'C', 'AJ', '104_ORP']
ntrl_NTU = ['B', 'C', 'AN', '104_NTU']
ntrl_FT = ['B', 'C', 'AW', '104_FT']

analysis_type_list = [ntrl_Pb, ntr_Zn, ntrl_Cu, ntrl_Ni, ntrl_Co, ntrl_Mn, ntrl_Cr, ntrl_Ca,
                ntrl_Si, ntrl_Al, ntrl_Mg, ntrl_Fe2, ntrl_pH, ntrl_ORP, ntrl_NTU, ntrl_FT]

currentmonthyr_list = [currentmonthyr_01, currentmonthyr_02, currentmonthyr_03, currentmonthyr_04,
                       currentmonthyr_05, currentmonthyr_06, currentmonthyr_07]

filepath_list = [filepath_01, filepath_02, filepath_03, filepath_04, filepath_05, filepath_06, filepath_07]

In [5]:
def generate_combined_df_by_month(month_WB, sample_area, analysis_type):
    df_List = []
    for i in analysis_type:
        df_analysis = month_WB.generate_df_from_analysis(sample_area, i)
        df_List.append(df_analysis)
    for i in df_List:
        df = join_DF(df_List)
    return df

In [ ]:
Mar_2020 = LabResults(currentmonthyr_03, filepath_03)
Mar2020_ntrl_df = generate_combined_df_by_month(Mar_2020, '104PU01', analysis_type_list)
Mar2020_ntrl_df.to_excel(r'C:\Users\v.t.flores\Documents\NTRL_Mar2020.xlsx')

In [7]:
Jan_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Jan2020.xlsx')
Feb_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Feb2020.xlsx')
Mar_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Mar2020.xlsx')
Apr_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Apr2020.xlsx')
May_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_May2020.xlsx')
Jun_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Jun2020.xlsx')
Jul_ntrl_2020_xl = pd.read_excel(r'C:\Users\v.t.flores\Documents\NTRL_Jul2020.xlsx')

ntrl_months_list = [Jan_ntrl_2020_xl, Feb_ntrl_2020_xl, Mar_ntrl_2020_xl, Apr_ntrl_2020_xl, May_ntrl_2020_xl, Jun_ntrl_2020_xl, Jul_ntrl_2020_xl]

In [14]:
NTRL_2020_DF = append_monthsDF(ntrl_months_list)
NTRL_2020_DF = NTRL_2020_DF.drop_duplicates(subset='DATETIME', keep="first")

In [15]:
NTRL_2020_DF.head()

,DATETIME,104_Pb,104_Zn,104_Cu,104_Ni,104_Co,104_Mn,104_Cr,104_Ca,104_Si,104_Al,104_Mg,104_Fe2,104_pH60,104_ORP,104_NTU,104_FT
0,2020-01-01 06:00:00,NaN,0.0704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9280,352.5,51.7,13.0
1,2020-01-01 07:00:00,0.0013,0.0883,0.0164,3.3040,0.3085,3.431,0.1643,0.6981,0.3119,4.736,5.078,1.625,2.9310,354.7,49.5,13.0
2,2020-01-01 08:00:00,NaN,0.0838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9380,350.1,50.3,13.0
3,2020-01-01 09:00:00,NaN,0.0789,NaN,3.3663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9310,349.4,47.9,12.0
4,2020-01-01 10:00:00,NaN,0.0840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9394,350.4,44.5,12.0


In [12]:
NTRL_2020_DF.to_excel(r'C:\Users\v.t.flores\Documents\NTRL_Jan_Jul2020.xlsx')

In [ ]:
#---------------spare code: for better understanding of the function generate_combined_df_by_month()
#
#
Jun_2020 = LabResults(currentmonthyr_06, filepath_06)
#NTRL Jun 2020
Jun2020_ntrl_Pb_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Pb)
Jun2020_ntrl_Zn_df = Jun_2020.generate_df_from_analysis('104PU01', ntr_Zn)
Jun2020_ntrl_Cu_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Cu)
Jun2020_ntrl_Ni_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Ni)
Jun2020_ntrl_Co_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Co)
Jun2020_ntrl_Mn_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Mn)
Jun2020_ntrl_Cr_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Cr)
Jun2020_ntrl_Ca_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Ca)
Jun2020_ntrl_Si_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Si)
Jun2020_ntrl_Al_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Al)
Jun2020_ntrl_Mg_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Mg)
Jun2020_ntrl_Fe2_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_Fe2)
Jun2020_ntrl_pH_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_pH)
Jun2020_ntrl_ORP_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_ORP)
Jun2020_ntrl_NTU_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_NTU)
Jun2020_ntrl_FT_df = Jun_2020.generate_df_from_analysis('104PU01', ntrl_FT)

dfsList6 = [Jun2020_ntrl_Pb_df, Jun2020_ntrl_Zn_df, 
            Jun2020_ntrl_Cu_df, Jun2020_ntrl_Ni_df,
            Jun2020_ntrl_Co_df, Jun2020_ntrl_Mn_df,
            Jun2020_ntrl_Cr_df, Jun2020_ntrl_Ca_df,
            Jun2020_ntrl_Si_df, Jun2020_ntrl_Al_df,
            Jun2020_ntrl_Mg_df, Jun2020_ntrl_Fe2_df,
            Jun2020_ntrl_pH_df, Jun2020_ntrl_ORP_df,
            Jun2020_ntrl_NTU_df, Jun2020_ntrl_FT_df]

Jun2020_ntrl_df = join_DF(dfsList6)
#
#
#---------------spare code: for better understanding of the function generate_combined_df_by_month()